In [9]:
import pandas as pd
import joblib
import pm4py
import numpy as np
import re 

In [2]:
data = joblib.load("../../data/processed/data_preprocessed.pkl")

In [71]:
data["event"] = np.where(data["activityType"] == "ECOMMERCE", data["actionType"], 
                         np.where((data["activityType"] == "EVENT") & (data["eventCategory"] == "video youtube"),  "video_" + data["eventAction"],
                                  np.where((data["activityType"] == "EVENT") & (data["eventCategory"] == "internal search"),  "INTERNAL_SEARCH",
                                           np.where(data["activityType"] == "EVENT", data["eventAction"], "LOAD_NEW_PAGE"))))
data["event"] = data["event"].str.rsplit(" - ", expand=True)[0] #delete seconds after the video event (all product videos)
data.loc[data["event"].str.contains("filter by",  case = False), "event"] = "FILTER_BY" #group filter events
data.loc[data["event"].str.contains("remov", case = False), "event"] = "REMOVE_FROM_CART" #group remove events
data.loc[data["event"].str.contains("compare", case = False), "event"] = "COMPARE_PRODUCTS" #group compare events
data.loc[data["event"].str.contains("add", case = False), "event"] = "ADD_TO_CART" #group add events
data.loc[data["event"].str.contains("download", case = False), "event"] = "DOWNLOAD" #group download events



In [110]:
page_categories = {
        "SERVICE" : ["retour-aanvragen", "service-verzoekformulier", "customer-service", "services", "service", 
                "webshop-retouraanvraag", "contacts", "contact", "thank-you-retour"],

        "LEGAL" : ["verkoopsvoorwaarden-algemeen", "algemene-verkoopsvoorwaarden", "faq", "privacy-policy", 
                "algemene-verkoopsvoorwaarden-particulier", "cookie-policy", "conditions-de-vente", 
                "conditions-de-vente-generales", "conditions-de-vente-generales-entreprise",
                "algemene-verkoopsvoorwaarden-aankoop-als-onderneming", 
                "conditions-de-vente-generales-consommateurs"],

        "PROMOTION" : ["promoties", "promotions", "bean-to-cup-gratis-cadeaubox", "lotus"],

        "COMPARISON" : ["producten-vergelijken", "comparaison-de-produits"],

        "COMPANY" : ["het-bedrijf", "entreprise", "Het-bedrijf"],

        "NEWS" : ["nieuws", "actualites", "newsletter"],

        "SEARCH" : ["zoekfunctie", "search"],

        "ACCOUNT" : ["activate", "my-account", "register", "forgot-password"],

        "JOBS" : ["jobs_product-manager", "jobs-chauffeur", "jobs_customer_service_medewerker", 
                "jobs-key-account-manager", "jobs-account-manager"],

        "SHOWROOM" : ["showroom-bezoek", "store-locator", "stores", "bezoek-showroom", 
                "showroom-visiteurs", "inplannen-showroom-bezoek"],

        "INFO" : ["info", "3dbestanden", "smeg-connect", "livrerecettesfourvapeur", 
                "app-leefkeuken", "stoomkookboek", "smegconnect", "gebruiksaanwijzingen", 
                "telecharger-les-catalogues"],

        "CHECKOUT" : ["bedankt", "merci", "wishlist", "cart", "checkout"],

        "INSPIRATION" : ["inspiratie-artikels", "Designlijnen", "inspiratie", "designlijnen", 
                "inspiration", "inspirations", "vitality", "designlijn", 
                "designs", "design", "samenwerkingen", "collaborations"],

        "ERROR" : ["pagenotfound"]
}


data["page"] = data["pagePath"].str.rsplit("/", expand=True)[2] 
data["pageExtra"] = data["pagePath"].str.rsplit("/", expand=True)[3] 
data["page"] = data["page"].str.rsplit("?", expand=True)[0] #delete parameters search
data["page"] = np.where(data["page"].isin(["service", "services"]) & 
                        data["pageExtra"].isin(["gebruiksaanwijzingen", "telecharger-les-catalogues"]), 
                         data["pageExtra"], data["page"])

data["page_new"] = np.nan
for key, values in page_categories.items(): 
      data.loc[data["page"].isin(values), "page_new"] = key

data.loc[(data["page"].notna() & data["page_new"].isna()), "page_new"] = "PRODUCT"
            

In [111]:
data.loc[data["page"].notna(), "event"] = data.loc[data["page"].notna(), "event"] + "_" + data.loc[data["page"].notna(), "event"]

In [105]:
pd.options.display.max_rows = 4000
print(data.loc[data["page_new"].isna(), "page"].value_counts())

Series([], Name: page, dtype: int64)


In [86]:
pd.options.display.max_rows = 4000
test = data["pagePath"].str.rsplit("/", expand=True)[3]

In [72]:
pd.options.display.max_rows = 4000
print(data["event"].value_counts())

LOAD_NEW_PAGE                  36788
video_progress                 20230
DETAILS_VIEW                   15945
video_start                     8770
video_complete                  6215
product page visualization      2762
CLICK                            869
click on a product               842
INTERNAL_SEARCH                  589
ADD_TO_CART                      380
FILTER_BY                        360
COMPARE_PRODUCTS                 112
DOWNLOAD                          76
click on internal promotion       50
CHECKOUT                          49
REMOVE_FROM_CART                  40
step2 view                        17
click on phone number             16
step1 option                      15
step1 view                        11
login                             10
account registration               7
PAYMENT                            5
successful order                   5
step2 option                       5
Name: event, dtype: int64


In [91]:
print(data["page"].value_counts())

product                                                 6359
produits                                                3207
customer-service                                        2458
promoties                                               1897
info                                                    1300
zoekfunctie                                             1094
sda-koffiemachines                                      1052
nieuws                                                   899
koelkasten                                               867
promotions                                               741
fornuizen                                                665
ovens                                                    605
products                                                 564
refrigerateurs                                           514
kookplaten                                               489
actualites                                               412
faq                     

productSku, pagina interesse, channelGrouping, platform, deviceCategory as filters 

In [ ]:
net, initial_marking, final_marking = pm4py.discover_petri_net_alpha_plus(data)
pm4py.write_petri_net(net, initial_marking, final_marking, "petri_net.pnml")
pm4py.save_vis_petri_net(net, initial_marking, final_marking, "petri_net.png")

In [ ]:
pm4py.write_xes(data, "../../data/processed/data_preprocessed.xes")